In [1]:
# Import base dependencies
import pandas as pd
import numpy as np
# Set view options for df
pd.set_option('display.max_columns', None)

In [2]:
# Read in raw data csv with some column dtype specifications
trulia = pd.read_csv("rawTruliaData/trulia_listings_2020-04-17 20:11:06.csv",
                    dtype={'url':'str','street_address':'str','city':'str','state':'str','zipcode':'str','neighborhood':'str','square_footage':'str'})
print(trulia.count())
trulia.head()

url               4640
street_address    4640
city              4640
state             4640
zipcode           4640
neighborhood      3098
beds              3671
baths             2915
square_footage    4168
price             4640
listing_type      4640
description       4634
details           4615
dtype: int64


,url,street_address,city,state,zipcode,neighborhood,beds,baths,square_footage,price,listing_type,description,details
0,https://www.trulia.com/p/ca/sacramento/3141-ye...,3141 Yellowstone Ln,Sacramento,CA,95821,NaN,4,2,"1,689","$390,000",BUY,"Beautifully upgrade, large 4 bedroom, 2 full b...","['Single Family Home', '$231/sqft', 'Lot Size:..."
1,https://www.trulia.com/p/ca/sacramento/7748-fi...,7748 Finnhorse Way,Sacramento,CA,95828,NaN,5,3,"3,020","$529,900",BUY,"Fabulous 5 bedroom, 3 bathroom, two-story prop...","['Single Family Home', '$175/sqft', 'Lot Size:..."
2,https://www.trulia.com/p/ca/rancho-cordova/224...,2246 Palmwood Ct,Rancho Cordova,CA,95670,Mills Ranch,4,2,"1,420","$368,900",BUY,"Beautifully updated 4 bedroom, 2 bath single s...","['Single Family Home', '$260/sqft', 'Lot Size:..."
3,https://www.trulia.com/p/ca/sacramento/5741-re...,5741 Rexleigh Dr,Sacramento,CA,95823,Valley High-North Laguna,4,3,"2,918","$470,000",BUY,"Beautiful 4 bedroom, 3 bathroom, 2 story locat...","['Single Family Home', '$161/sqft', 'Lot Size:..."
4,https://www.trulia.com/p/ca/sacramento/6101-go...,6101 Golden Dawn Way,Sacramento,CA,95841,Foothill Farms,4,3,"1,903","$375,000",BUY,This home has an attached in-laws unit with it...,"['Single Family Home', '$197/sqft', 'Lot Size:..."


In [3]:
# Remove any duplicate rows of data
trulia = trulia.drop_duplicates()
trulia.count()

url               3342
street_address    3342
city              3342
state             3342
zipcode           3342
neighborhood      2164
beds              2827
baths             2202
square_footage    3023
price             3342
listing_type      3342
description       3336
details           3341
dtype: int64

In [4]:
# Clean up beds, baths, square_footage, and price for dtype conversion
# These functions were created after analyzing the values for each column
def bed_conv(beds):
    if beds in ['1-2','Studio-2']:
        return 2
    elif beds in ['1-3','Studio-3','2-3']:
        return 3
    elif beds in ['1-4','Studio-4']:
        return 4
    elif beds in ['S','St','Studio-']:
        return 1
    else:
        return beds
    
def bath_conv(baths):
    if baths in ['1-2','1.5-2']:
        return 2
    elif baths == '1-2.5':
        return 2.5
    elif baths == '1-1.5':
        return 1.5
    elif baths == '1-3':
        return 3
    elif baths == '1-3.5':
        return 3.5
    elif baths == '0+':
        return 1
    else:
        return baths

def sq_cleanup(sq):
    try:
        if '-' in sq:
            return sq.split('-')[1].replace(',','')
        elif sq in ['1','2']:
            return np.nan
        else:
            return sq.replace(',','')
    except Exception as e:
        code = 0
        
def price_cleanup(price):
    try:
        if 'C' in price:
            return np.nan
        elif '-' in price:
            return price.split('-').strip('$/mo+').replace(',','')
        else:
            return price.strip('$/mo+').replace(',','')
    except Exception as e:
        code = 0
        
    
for index,row in trulia.iterrows():
    trulia['beds'][index] = bed_conv(row['beds'])
    trulia['baths'][index] = bath_conv(row['baths'])
    trulia['square_footage'][index] = sq_cleanup(row['square_footage'])
    trulia['price'][index] = price_cleanup(row['price'])
    if row['city'] == 'Rncho Cordova':
        trulia['city'][index] = 'Rancho Cordova'

# Remove any rows without price data
trulia = trulia.dropna(subset=['price'])
trulia.head()

,url,street_address,city,state,zipcode,neighborhood,beds,baths,square_footage,price,listing_type,description,details
0,https://www.trulia.com/p/ca/sacramento/3141-ye...,3141 Yellowstone Ln,Sacramento,CA,95821,NaN,4,2,1689,390000,BUY,"Beautifully upgrade, large 4 bedroom, 2 full b...","['Single Family Home', '$231/sqft', 'Lot Size:..."
1,https://www.trulia.com/p/ca/sacramento/7748-fi...,7748 Finnhorse Way,Sacramento,CA,95828,NaN,5,3,3020,529900,BUY,"Fabulous 5 bedroom, 3 bathroom, two-story prop...","['Single Family Home', '$175/sqft', 'Lot Size:..."
2,https://www.trulia.com/p/ca/rancho-cordova/224...,2246 Palmwood Ct,Rancho Cordova,CA,95670,Mills Ranch,4,2,1420,368900,BUY,"Beautifully updated 4 bedroom, 2 bath single s...","['Single Family Home', '$260/sqft', 'Lot Size:..."
3,https://www.trulia.com/p/ca/sacramento/5741-re...,5741 Rexleigh Dr,Sacramento,CA,95823,Valley High-North Laguna,4,3,2918,470000,BUY,"Beautiful 4 bedroom, 3 bathroom, 2 story locat...","['Single Family Home', '$161/sqft', 'Lot Size:..."
4,https://www.trulia.com/p/ca/sacramento/6101-go...,6101 Golden Dawn Way,Sacramento,CA,95841,Foothill Farms,4,3,1903,375000,BUY,This home has an attached in-laws unit with it...,"['Single Family Home', '$197/sqft', 'Lot Size:..."


In [5]:
trulia.count()

url               3158
street_address    3158
city              3158
state             3158
zipcode           3158
neighborhood      2053
beds              2671
baths             2096
square_footage    2861
price             3158
listing_type      3158
description       3152
details           3158
dtype: int64

In [6]:
# Convert dtypes
trulia.astype({'beds':'float32','baths':'float32','square_footage':'float32','price':'float32'}).dtypes

url                object
street_address     object
city               object
state              object
zipcode            object
neighborhood       object
beds              float32
baths             float32
square_footage    float32
price             float32
listing_type       object
description        object
details            object
dtype: object

In [7]:
# Create deep copies for separate buy/rent df's
buys = trulia.loc[trulia['listing_type'] == 'BUY',:].copy()
rents = trulia.loc[trulia['listing_type'] == 'RENT',:].copy()

In [8]:
# Get unique zipcodes and neighborhoods for later use
print(trulia['city'].unique())
# print(trulia['zipcode'].unique())

['Sacramento' 'Rancho Cordova' 'Folsom' 'Citrus Heights' 'Antelope'
 'Elk Grove' 'West Sacramento' 'Orangevale' 'Rio Linda' 'Gold River'
 'Fair Oaks' 'Carmichael' 'North Highlands' 'Mather' 'Wilton'
 'Sloughhouse' 'Clarksburg' 'Elverta' 'Nimbus']


In [9]:
buys.details[16]

"['Single Family Home', '$194/sqft', 'Lot Size: 6,364 sqft', '$71/monthly HOA', 'Built in 2000', '6 Days on Trulia', '7 Rooms', 'Rooms: Dining Room, Office, Walk In Closet', 'Heating: Forced Air', 'See Virtual Tour', 'Cooling System: Central', 'Air Conditioning', 'Refrigerator', 'Microwave', 'Dishwasher', 'Disposal', 'Floors: Carpet, Tile', 'Fireplace', 'Double Paned Windows', 'Ceiling Fan', 'Parking: Attached Garage', 'Garage', 'Parking Spaces: 3', 'Fitness Center', 'Cable Ready', 'Contemporary Architecture', 'Stories: 2', 'Exterior: Stone Stucco Wood', 'Foundation Type: Slab', 'Roof: Composition', 'Patio', 'Porch', 'Pond', 'Waterfront', 'MLS/Source ID: 20021221']"

In [10]:
# Define variables for unique values of each 'list' feature
# These values were obtained through earlier analysis of the data
unique_values = {'rooms': ['Dining Room','Family Room','Laundry Room','Walk In Closet','Pantry','Breakfast Nook',
        'Office','Workshop','Library','Recreation Room','Master Bedroom','Living Room','Loft'],
    'cool': ['Central', 'Evaporative', 'Wall', 'None', 'Solar', 'Refrigeration'],
    'floors': ['Carpet','Hardwood','Laminate','Tile','Linoleum Vinyl','Concrete','Slate'],
    'parking': ['Detached Garage','Attached Garage','Carport','Off Street','On Street','None','Garage Attached'],
    'exteriors': ['Brick','Wood','Stucco','Stone','Cement','Concrete','Vinyl','Metal','Shingle','Composition',
        'Products'],
    'pets': ['No pets allowed', 'Cats allowed', 'Small dogs allowed', 'large dogs allowed'],
    'rparking': ['On Street', 'Off Street', 'Garage Detached', 'Garage Attached'],
    'util': ['Sewage','Garbage','Water','Hot Water','Internet','Cable','Electricity','Gas','Heat','Phone']
    }

In [11]:
# Add empty columns for BUY features data
buy_features = ['home_type','lot_size','year_built','num_rooms','heating', 'heating_fuel','air_con','microwave',
                'dishwasher','disposal','washer','dryer','fireplace','vaulted_ceiling','double_pane','garage',
                'num_parking','security','num_stories','foundation_type','patio','porch','deck','pool']
for lst in ['rooms', 'cool', 'floors', 'parking', 'exteriors']:
    for col in unique_values[lst]:
        buy_features.append(f'{lst}_{col}')
for f in buy_features:
    buys[f] = np.nan

In [12]:
# Import dependencies for 'details' parsing
import ast
import itertools
home_types = ['Lot Land', 'Condo', 'Multi Family', 'Townhouse', 'Single Family Home', 'Apartment', 'Mobile Manufactured']

In [13]:
# Parse 'details' and log all data into respective columns
for index,row in buys.iterrows():
    try:
        # Cast 'details' as list for parsing
        details = ast.literal_eval(row['details'])
        for feature in details:
#             print(feature)
            if feature in home_types:
                buys['home_type'][index] = feature
                continue
            if 'Lot Size' in feature:
                if 'sqft' in feature:
                    buys['lot_size'][index] = int(feature[10:-5].replace(',',''))
                else:
                    buys['lot_size'][index] = float(feature[10:-6])*43560 # convert acre to sqft
                continue
            if 'Built in' in feature:
                buys['year_built'][index] = int(feature[8:])
                continue
            if ' Rooms' in feature:
                buys['num_rooms'][index] = int(feature[:-6])
                continue
            if 'Rooms:' in feature:
                for rm in [room.strip() for room in feature[7:].split(',')]:
                    if rm in unique_values['rooms']:
                        buys[f'rooms_{rm}'][index] = rm
                continue
            if 'Heating Fuel:' in feature:
                buys['heating_fuel'][index] = feature[13:]
                continue
            if 'Heating:' in feature:
                buys['heating'][index] = feature[9:]
                continue
            if 'Cooling' in feature:
                for cl in [cool.strip() for cool in feature[15:].split(',')]:
                    if cl in unique_values['cool']:
                        buys[f'cool_{cl}'][index] = cl
                continue
            if 'Floors:' in feature:
                for flr in [floor.strip() for floor in feature[8:].split(',')]:
                    if flr in unique_values['floors']:
                        buys[f'floors_{flr}'][index] = flr
                continue
            if 'Air Con' in feature:
                buys['air_con'][index] = feature
                continue
            if 'Microwave' in feature:
                buys['microwave'][index] = feature
                continue
            if 'Dishwasher' in feature:
                buys['dishwasher'][index] = feature
                continue
            if 'Disposal' in feature:
                buys['disposal'][index] = feature
                continue
            if 'Washer' in feature:
                buys['washer'][index] = feature
                continue
            if 'Dryer' in feature:
                buys['dryer'][index] = feature
                continue
            if 'Fireplace' in feature:
                buys['fireplace'][index] = feature
                continue
            if 'Vaulted' in feature:
                buys['vaulted_ceiling'][index] = feature
                continue
            if 'Double Pan' in feature:
                buys['double_pane'][index] = feature
                continue
            if 'Parking Spaces:' in feature:
                buys['num_parking'][index] = int(feature[15:])
                continue
            if 'Parking:' in feature:
                for prk in [park.strip() for park in feature[9:].split(',')]:
                    if prk in unique_values['parking']:
                        buys[f'parking_{prk}'][index] = prk
                continue
            if 'Garage' in feature:
                buys['garage'][index] = feature
                continue
            if 'Security' in feature:
                buys['security'][index] = feature
                continue
            if 'Stories:' in feature:
                buys['num_stories'][index] = int(feature[8:])
                continue
            if 'Exterior' in feature:
                for exterior in [ext.strip() for ext in feature[10:].split(' ')]:
                    if exterior in unique_values['exteriors']:
                        buys[f'exteriors_{exterior}'][index] = exterior
                continue
            if 'Foundation' in feature:
                buys['foundation_type'][index] = feature[16:]
                continue
            if 'Roof' in feature:
                buys['roof_type'][index] = feature[6:]
                continue
            if 'Patio' in feature:
                buys['patio'][index] = feature
                continue
            if 'Porch' in feature:
                buys['porch'][index] = feature
                continue
            if 'Deck' in feature:
                buys['deck'][index] = feature
                continue
            if 'Pool' in feature:
                buys['pool'][index] = feature
                continue
    except Exception as e:
#         print(f'Error on i:{index}, {e}') 
        code = 0

/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/JLow/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f

/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:106:

In [14]:
for feature in buy_features:
    print(buys[feature].value_counts())
    print('----------------------------')

Single Family Home     1487
Lot Land                207
Condo                   201
Mobile Manufactured     106
Multi Family            106
Townhouse                39
Apartment                 3
Name: home_type, dtype: int64
----------------------------
6098.0      76
6534.0      69
7405.0      52
6970.0      42
5227.0      34
            ..
8540.0       1
258746.4     1
911275.2     1
6390.0       1
1392.0       1
Name: lot_size, Length: 868, dtype: int64
----------------------------
2018.0    43
2017.0    39
1959.0    39
1960.0    38
1950.0    38
          ..
1906.0     1
1890.0     1
1943.0     1
1918.0     1
1912.0     1
Name: year_built, Length: 113, dtype: int64
----------------------------
6.0     312
5.0     308
7.0     225
8.0     173
4.0     113
9.0      82
2.0      44
10.0     38
3.0      29
11.0     21
12.0     18
16.0     12
14.0      6
13.0      6
1.0       3
15.0      2
20.0      1
18.0      1
21.0      1
Name: num_rooms, dtype: int64
----------------------------
Forced

In [15]:
buys.count()

url                      2151
street_address           2151
city                     2151
state                    2151
zipcode                  2151
                         ... 
exteriors_Vinyl            54
exteriors_Metal            29
exteriors_Shingle           6
exteriors_Composition       4
exteriors_Products          5
Length: 81, dtype: int64

In [16]:
# Add empty columns for RENT features data
rent_features = ['home_type','year_built','deposit','smoking',
                 'living_room','dishwasher','microwave','refrigerator','on_site_maint','on_site_mng','laundry',
                 'air_con','disposal','dryer','patio','pool','balcony','washer','basketball','ceiling_fan',
                 'fireplace','fitness','playground','floor_types','bbq','vaulted_ceiling']
for lst in ['rparking', 'pets', 'util']:
    for col in unique_values[lst]:
        rent_features.append(f'{lst}_{col}')
for f in rent_features:
    rents[f] = np.nan

In [17]:
# Parse 'details' for rents

def pet_filter(pet_str):
    if 'Small' in  pet_str or 'small' in pet_str:
        return 'Small dogs allowed'
    elif 'Cats' in pet_str:
        return 'Cats allowed'
    else:
        return pet_str.strip()
    
for index,row in rents.iterrows():
    try:
        details = ast.literal_eval(row['details'])
        for feature in details:
#             print(feature)
            if feature in home_types:
                rents['home_type'][index] = feature
                continue
            if 'Built in' in feature:
                rents['year_built'][index] = int(feature[8:])
                continue
            if 'allowed' in feature:
                for pt in [pet_filter(pet) for pet in feature.split(',')]:
                    if pt in unique_values['pets']:
                        rents[f'pets_{pt}'][index] = pt
                continue
            if 'Deposit' in feature:
                rents['deposit'][index] = int(feature[10:].replace(',',''))
                continue
            if 'Rent Includes' in feature:
                for utility in [util.strip() for util in feature[15:].split(',')]:
                    if utility in  unique_values['util']:
                        rents[f'util_{utility}'][index] = utility
                continue
            if 'Floors:' in feature:
                rents['floor_types'][index] = feature[8:]
                continue
            if 'Air Con' in feature:
                rents['air_con'][index] = feature
                continue
            if 'Microwave' in feature:
                rents['microwave'][index] = feature
                continue
            if 'Dishwasher' in feature:
                rents['dishwasher'][index] = feature
                continue
            if 'Disposal' in feature:
                rents['disposal'][index] = 'Disposal'
                continue
            if 'Washer' in feature:
                rents['washer'][index] = feature
                continue
            if 'Dryer' in feature:
                rents['dryer'][index] = feature
                continue
            if 'Fireplace' in feature:
                rents['fireplace'][index] = 'Fireplace'
                continue
            if 'smoking' in feature:
                rents['smoking'][index] = 'No smoking'
                continue
            if 'Living room' in feature:
                rents['living_room'][index] = feature
                continue
            if 'Management' in feature:
                rents['on_site_mng'][index] = feature
                continue
            if 'Maintenance' in feature and 'On' in feature:
                rents['on_site_maint'][index] = feature
                continue
            if 'Parking:' in feature:
                for prk in [park.strip() for park in feature[9:].split(',')]:
                    if prk in unique_values['rparking']:
                        rents[f'rparking_{prk}'][index] = prk
                continue
            if 'Ceiling Fan' in feature:
                rents['ceiling_fan'][index] = feature
                continue
            if 'Vault' in feature:
                rents['vaulted_ceiling'][index] = 'Vaulted Ceilings'
                continue
            if 'Laundry' in feature:
                if ':' in feature:
                    rents['laundry'][index] = feature[9:]
                else:
                    rents['laundry'][index] = feature
                continue
            if 'Basketball' in feature:
                rents['basketball'][index] = feature
                continue
            if 'Patio' in feature:
                rents['patio'][index] = 'Patio'
                continue
            if 'Balcony' in feature:
                rents['balcony'][index] = feature
                continue
            if 'Refrigerator' in feature:
                rents['refrigerator'][index] = 'Refrigerator'
                continue
            if 'Pool' in feature:
                rents['pool'][index] = feature
                continue
            if 'Fitness' in feature:
                rents['fitness'][index] = feature
                continue
            if 'Playground' in feature:
                rents['playground'][index] = feature
                continue
            if 'Area' in feature and ('Bar' in feature or 'BB' in feature):
                rents['bbq'][index] = 'BBQ Area'
                continue
    except Exception as e:
#         print(f'Error on i:{index}, {e}')            
        code = 0

/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: Set

/Users/JLow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
for feature in rent_features:
    print(rents[feature].value_counts())
    print('----------------------------')

Multi Family          445
Single Family Home    442
Apartment              78
Townhouse              42
Name: home_type, dtype: int64
----------------------------
1972.0    19
2005.0    18
1977.0    17
1960.0    15
1964.0    15
          ..
2014.0     1
1918.0     1
1916.0     1
1931.0     1
1949.0     1
Name: year_built, Length: 108, dtype: int64
----------------------------
1000.0    49
2000.0    40
1500.0    32
500.0     23
2500.0    21
          ..
1145.0     1
2375.0     1
1970.0     1
2175.0     1
1498.0     1
Name: deposit, Length: 159, dtype: int64
----------------------------
No smoking    39
Name: smoking, dtype: int64
----------------------------
Living room    77
Name: living_room, dtype: int64
----------------------------
Dishwasher    239
Name: dishwasher, dtype: int64
----------------------------
Microwave    143
Name: microwave, dtype: int64
----------------------------
Refrigerator    258
Name: refrigerator, dtype: int64
----------------------------
Online Maintenance 

In [19]:
rents.count()

url                         1007
street_address              1007
city                        1007
state                       1007
zipcode                     1007
neighborhood                 754
beds                         749
baths                        463
square_footage               886
price                       1007
listing_type                1007
description                 1007
details                     1007
home_type                   1007
year_built                   593
deposit                      863
smoking                       39
living_room                   77
dishwasher                   239
microwave                    143
refrigerator                 258
on_site_maint                  6
on_site_mng                    7
laundry                      651
air_con                      540
disposal                     165
dryer                        370
patio                          4
pool                          84
balcony                      291
washer    

In [20]:
# Impute missing values for numerical data (median)
# Define lists of numerical columns 
buys_num = ['beds','baths','square_footage','lot_size','year_built','num_rooms','num_parking','num_stories']
rents_num = ['beds','baths','square_footage','year_built','deposit']
# Instantiate dictionaries for the medians
buys_medians = {}
rents_medians = {}

In [21]:
# Find medians for each of the numerics in BUYS
for col in buys_num:
    no_nan = buys[col].dropna()
    buys_medians[col] = no_nan.median()
buys_medians

{'beds': 3.0,
 'baths': 2.5,
 'square_footage': 1626.0,
 'lot_size': 6534.0,
 'year_built': 1975.0,
 'num_rooms': 6.0,
 'num_parking': 2.0,
 'num_stories': 1.0}

In [22]:
# Find medians for each of the numerics in RENTS
for col in rents_num:
    no_nan = rents[col].dropna()
    rents_medians[col] = no_nan.median()
rents_medians

{'beds': 2.0,
 'baths': 2.0,
 'square_footage': 1100.0,
 'year_built': 1972.0,
 'deposit': 1600.0}

In [23]:
# Impute null data with medians
buys = buys.fillna(value=buys_medians)
rents = rents.fillna(value=rents_medians)

In [24]:
# Check counts of imputed columns
rents.count()

url                         1007
street_address              1007
city                        1007
state                       1007
zipcode                     1007
neighborhood                 754
beds                        1007
baths                       1007
square_footage              1007
price                       1007
listing_type                1007
description                 1007
details                     1007
home_type                   1007
year_built                  1007
deposit                     1007
smoking                       39
living_room                   77
dishwasher                   239
microwave                    143
refrigerator                 258
on_site_maint                  6
on_site_mng                    7
laundry                      651
air_con                      540
disposal                     165
dryer                        370
patio                          4
pool                          84
balcony                      291
washer    

In [25]:
buys.count()

url                      2151
street_address           2151
city                     2151
state                    2151
zipcode                  2151
                         ... 
exteriors_Vinyl            54
exteriors_Metal            29
exteriors_Shingle           6
exteriors_Composition       4
exteriors_Products          5
Length: 81, dtype: int64

In [26]:
# Export cleaned data
rents.to_csv('clean_rents.csv')
buys.to_csv('clean_buys.csv')
print('Export Complete')

Export Complete
